In [2]:
import operator
import re
import string
import warnings
from os import path

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from corextopic import corextopic as ct
from nltk.corpus import stopwords
from PIL import Image
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import STOPWORDS, ImageColorGenerator, WordCloud

nltk.download('stopwords')
%matplotlib inline
sns.set_style('whitegrid')
%matplotlib inline
warnings.simplefilter("ignore", DeprecationWarning)

[nltk_data] Downloading package stopwords to /Users/dhara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
barley=pd.read_csv('../Dataset/barley_no_zero_duplicates.csv')
#bean=pd.read_csv('bean_no_zero_duplicates.csv')
corn_maize=pd.read_csv('../Dataset/corn_maize_no_zero_duplicates.csv')
#edamame=pd.read_csv('edamame_no_zero_duplicates.csv')
#lentil=pd.read_csv('lentil_no_zero_duplicates.csv')
millet=pd.read_csv('../Dataset/millet_no_zero_duplicates.csv')
#nut=pd.read_csv('nut_no_zero_duplicates.csv')
oat=pd.read_csv('../Dataset/oat_no_zero_duplicates.csv')
#pea=pd.read_csv('pea_no_zero_duplicates.csv')
potato=pd.read_csv('../Dataset/potato_no_zero_duplicates.csv')
quinoa=pd.read_csv('../Dataset/quinoa_no_zero_duplicates.csv')
rice=pd.read_csv('../Dataset/rice_no_zero_duplicates.csv')
rye=pd.read_csv('../Dataset/rye_no_zero_duplicates.csv')
sorghum=pd.read_csv('../Dataset/sorghum_no_zero_duplicates.csv')
soybean=pd.read_csv('../Dataset/soybean_no_zero_duplicates.csv')
wheat=pd.read_csv('../Dataset/wheat_no_zero_duplicates.csv')
legume_nuts_no_peanuts_no_zero_duplicates=pd.read_csv('../Dataset/legume_no_zero_duplicates.csv')
legume_peanuts_no_zero_duplicates=pd.read_csv('../Dataset/legume_peanuts_no_zero_duplicates.csv')
legume_other=pd.read_csv('../Dataset/legume_other_no_zero_duplicates.csv')
whole=pd.read_csv('../Dataset/all_no_zero_duplicates.csv')

In [9]:
whole.shape

(10637, 26)

barley['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']

corn_maize['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']

millet['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']

oat['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']

potato['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
quinoa['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
rice['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
rye['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
sorghum['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
soybean['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
wheat['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
legume['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']
whole['TEXT']=barley['ABSTRACT']+barley['PROJECT_TERMS']+barley['PROJECT_TITLE']

# Combined analysis for all the crops

In [35]:
wheat.head()

,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ABSTRACT
0,584433,Anabolism; antiporter; Arabidopsis; Biochemis...,"PHYTONUTRIENT BIOCHEMISTRY, PHYSIOLOGY, AND TR...",USDA,ARS,NaN,ARS-0408087,5/21/2004,3/31/2009,"UPCHURCH, DAN R",...,TX,77030,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): 1) Identify and characterize pla...
1,584463,Affect; Biochemical; Bread; Breeding; Carbohy...,ENHANCE HARD SPRING AND DURUM WHEAT QUALITY AN...,USDA,ARS,NaN,ARS-0408465,9/10/2004,7/5/2009,"HARELAND, GARY A",...,ND,58102-2765,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Evaluate the processing and end-...
2,584455,Avena sativa; biosecurity; Breeding; Cereals;...,OBJECTIVE GRADING AND END-USE PROPERTY ASSESSM...,USDA,ARS,NaN,ARS-0408432,9/25/2004,7/5/2009,"PEARSON, THOMAS CLARK",...,KS,66502,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Develop instrumentation and proc...
3,584464,Amino Acids; Amylopectin; Amylose; base; Bioc...,OPTICAL PROPERTIES OF SMALL GRAINS FOR RAPID A...,USDA,ARS,NaN,ARS-0408471,9/15/2004,9/3/2009,"DELWICHE, STEPHEN R",...,MD,20705,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): (1) Develop an automated system ...
4,584523,Architecture; Bioinformatics; Cell Wall; Cere...,BIOTECHNOLOGICAL ENHANCEMENT OF ENERGY CROPS,USDA,ARS,NaN,ARS-0408875,8/15/2004,8/14/2009,"ANDERSON, OLIN D",...,CA,94710,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Develop the necessary range of k...


In [36]:
wheat.shape

(1319, 25)

## With Anchor Words

In [4]:
dataframe = barley.copy()
print('barley')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)

barley


In [6]:
import pandas as pd
df=pd.read_csv('Topics_corex/barley.csv')
df.head()

,Unnamed: 0,TEXT,0,1,2,3,4,5,6
0,0,Objective(s): 1) To assess the ability of barl...,0.000001,0.000001,0.000001,0.999989,0.000001,0.999962,0.000001
1,1,Objective(s): Enhance value and utilization of...,0.000001,0.000001,0.000001,0.000001,0.999999,0.000001,0.999998
2,2,Objective(s): Lower the cost of fuel ethanol p...,0.999999,0.000001,0.999999,0.000001,0.999999,0.999999,0.000001
3,3,Objective(s): Objective of this research are; ...,0.999999,0.000001,0.999999,0.999999,0.999999,0.999999,0.000001
4,4,Objective(s): Improve the functional propertie...,0.000001,0.000001,0.000001,0.000001,0.999999,0.000001,0.000001


In [8]:
dataframe['TEXT_PROCESSED'][2]

"objective(s): lower the cost of fuel ethanol production from corn and barley through improved dry grind and dry milling fractionation techniques including a new 'ammoniation' process develop more efficient processes for converting hulled and hulless barley to fuel ethanol and improved beta-glucan-free feed coproducts  through research assist in creation of a new hulless barley-to-ethanol industry in corn deficient regions particularly the mid atlantic states and the north western us  use the low-starch ('low carb') and high-fiber high-oil and high-protein fractions recovered from corn and barley prior to ethanol fermentation to produce health-promoting food ingredients functional foods and extruded snacks  develop improved processes to convert low valued crop-related biomass byproducts and energy crops being researched in the ars energy crop program into renewable hydrogen or liquid fuels and conduct economic feasibility studies for integrating this technology into co-located dry grin

In [ ]:
economics, Production, biological stress, Environment, Cultivation and Management, Animals and Non-human applications, human health

In [5]:
dataframe = barley.copy()
print('barley')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/barley.csv')

barley
Topic #1: production, yield, harvest, growth, yields, productivity, crop, quality, growth development, plant growth
Topic #2: resistance, diseases, disease, pathogen, pest, virus, pests, pesticides, fungus, host
Topic #3: environment, soil, climate, environmental, climate change, farming, adaptation, farming environment, gases, farm
Topic #4: nutrient, cropping, rotation, nitrogen, nutrients, water, rotations, fertilizer, tillage, cropping systems
Topic #5: diet, nutrition, diets, human health, pisum, pisum sativum, sativum, dietary, dry, lentils dietary
Topic #6: feed, animal, animals, livestock, feeding, biofuels, cattle, feeding food, human, humans
Topic #7: genes, gene, genes genetic, genetic, identify, objective, genome


In [38]:
dataframe = corn_maize.copy()
print('corn_maize')

dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/corn_maize.csv')

corn_maize
Topic #1: students, training, student, undergraduate, graduate, educational, university, graduate student, postdoctoral, universities
Topic #2: resistance, pest, pests, disease, diseases, pathogen, fungus, pesticides, fungi, insecta
Topic #3: climate, soil, climate change, environmental, gases, greenhouse gas, environment, ecosystem services, mitigation, change
Topic #4: cropping, rotation, nitrogen, water, fertilizer, nutrient, tillage, rotations, cover crop, conservation
Topic #5: diet, diets, nutrition, human health, public health, food quality, health, grains, fed, nutritional
Topic #6: feed, biofuels, biofuel, animal, cattle, livestock, animals, ethanol, dairy, milk
Topic #7: genes, maize, genetic, gene, genes genetic, genome, genomics, molecular, expression


In [39]:
dataframe = millet.copy()
print('millet')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/millet.csv')

millet
Topic #1: yield, growth, one, major, based, area, non, value, number, new
Topic #2: stress, resistance, disease, host, diverse, gene, strategies
Topic #3: environment, environmental, provide
Topic #4: animal, focus, may, many, used, systems, addition, grass
Topic #5: economics, economic, well, farmer, farmers, programs, year, state, years
Topic #6: students, training, student, undergraduate, analysis, educational, graduate, three, contribute, development
Topic #7: resources, universities, resource, address, methods, products, first, data, impact


In [40]:
dataframe = oat.copy()
print('oat')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/oat.csv')

oat
Topic #1: yield, productivity, production, harvest, yields, production productivity, quality, management, planting, yield quality
Topic #2: resistance, disease, diseases, rust, pathogen, germplasm, resistance resistance, genetic, genetic genetic, marker
Topic #3: environment, soil, environmental, farming, farming environment, farm, soybeans, crops, farms, agricultural
Topic #4: nutrient, rotation, cropping, nutrients, cover crop, water, cover, cropping systems, cover crops, different
Topic #5: diet, diets, determine effect, effect, fatty, proteins, protein, components, functionality, liquid
Topic #6: feed, animal, animals, livestock, feeding, environment feeding, animals avena, alternative, life, system testing
Topic #7: economics, markets, losses, economic, billion, beneficial, state, products, significant


In [41]:
dataframe = legume.copy()
print('legume')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/legume.csv')

legume
Topic #1: students, training, student, undergraduate, graduate, science, school
Topic #2: resistance, pests, disease, pest, diseases, pathogen, pesticides, virus, pesticide, fungus
Topic #3: soil, environmental, environment, climate, climate change, gases, ecosystem services, greenhouse gas, management, ecosystem
Topic #4: nitrogen, rotation, fertilizer, cropping, nutrient, rotations, cover crop, tillage, water, fabaceae
Topic #5: diet, human health, diets, public health, food, health, human, food food, nutrition, foods
Topic #6: animals, animal, livestock, cattle, feed, dairy, meat, milk, grazing, forage
Topic #7: economics, markets, farming environment, farming, economic, growers, farm


In [42]:
dataframe = potato.copy()
print('potato')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/potato.csv')

potato
Topic #1: production, yield, yields, growth, productivity, process production, growers, potato production, harvest, production programs
Topic #2: pathogen, resistance, diseases, virus, pests, disease, pest, pesticides, fungus, pesticide
Topic #3: environment, environmental, climate, climate change, farming environment, farming, soil, farm, gases, change
Topic #4: water, nutrient, rotation, nitrogen, nutrients, rotations, fertilizer, cropping, crop rotation, fertilizers
Topic #5: diet, nutrition, human health, diets, food quality, food, foods, health, products, vegetables
Topic #6: animal, animals, training, students, meat, livestock, student, educational, feed, graduate
Topic #7: genes, gene, genetic, molecular, genes genetic, expression, sequence, genome


In [43]:
dataframe = quinoa.copy()
print('quinoa')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=1,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/quinoa.csv')

quinoa
Topic #1: growth, yield, agricultural, agriculture, harvest, increasing, eastern, human
Topic #2: stress, pathogen, tolerance, disease, diseases, resistance, representative, regarding, ability, farms
Topic #3: adaptation, environment, early, three, important crop, chenopodium, america, agricultural practices, crops, practices
Topic #4: abiotic, sequencing genomics, markers generated, sequencing targeted, management hawaii, levels including, shifts, lead methods, lead diversity, worthy support
Topic #5: diet, acid, well, diets, pathway, acids, biological, tyrosine, application, enzymes
Topic #6: one, important, tools, tool, livestock, animals, feed, world, part, around
Topic #7: economics, economic, workshop, measures, educational workshop, economics educational, future, critical, organic, growers


In [44]:
dataframe = rice.copy()
print('rice')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/rice.csv')

rice
Topic #1: rice university, university, students, universities, research research
Topic #2: stress, pathogen, resistance, tolerance, disease, genes, gene, plant, plants, fungus
Topic #3: soil, climate, environmental, climate change, adaptation, environment, gases, greenhouse gas, soils, agriculture
Topic #4: water, nutrient, nutrients, nitrogen, fertilizer, cropping, tillage, rotation, fertilizers, water water
Topic #5: food security, nutrition, public health, diet, human health, food, health, food quality, food food, security
Topic #6: animals, animal, feed, biofuel, fish, livestock, biofuels, cattle, dairy, milk
Topic #7: economics, markets, economic, marketing, management


In [45]:
dataframe = rye.copy()
print('rye')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/rye.csv')

rye
Topic #1: yield, productivity, harvest, practice, production productivity, management practices, practice management, growth, agricultural, practices
Topic #2: pathogen, disease, pest, diseases, resistance, pathogens, grown, insect, oat, sequences
Topic #3: environmental, climate, impacts, corn, model, modeling, soybean, surface system, components, surface
Topic #4: cover crop, cropping, rotation, nutrient, rotations, nitrogen, nutrients, tillage, water, fertilizer
Topic #5: diet, feeding, environment feeding, characterize, performance, land, potential, related, state, mixtures
Topic #6: animal, animals, cattle, feed, livestock, producers, beef, goals, diversity
Topic #7: economics, economic, efficiency, forage, alternative, cereal rye, objective evaluate, farm, sub objective, planting


In [46]:
dataframe = sorghum.copy()
print('sorghum')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/sorghum.csv')

sorghum
Topic #1: productivity, yield, harvest, yields, growth, production productivity, productivity programs, tolerant, biomass breeding, trials
Topic #2: resistance, disease, pests, pest, pathogen, diseases, virus, fungus, weeds, stress
Topic #3: environmental, climate, soil, environment, gases, climate change, greenhouse gas, adaptation, change, carbon
Topic #4: cropping, rotation, tillage, fertilizer, water, nitrogen, nutrient, conservation, crop rotation, cover crop
Topic #5: diet, food security, nutrition, diets, students, training, student, undergraduate, university, function
Topic #6: biofuels, biofuel, feed, animal, animals, livestock, cattle, dairy, conversion, energy
Topic #7: genes, genetic, gene, genes genetic, genome, identify, genomics, germplasm, genomic


In [47]:
dataframe = soybean.copy()
print('soybean')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/soybean.csv')

soybean
Topic #1: yield, yields, harvest, productivity, growth, increase, field, acreage, planting, production productivity
Topic #2: resistance, pest, pests, pathogen, disease, diseases, pesticides, virus, weeds, fungus
Topic #3: environmental, soil, climate, climate change, environment, gases, greenhouse gas, ecosystem services, change, impacts
Topic #4: rotation, cropping, tillage, water, fertilizer, nutrient, rotations, conservation, nitrogen, cover crop
Topic #5: diet, diets, nutrition, plant based, nutritional, soy, development diet, human nutrition, dietary, malignant
Topic #6: feed, animal, animals, livestock, biofuels, cattle, fish, biofuel, dairy, meat
Topic #7: gene, genes, genetic, molecular, expression, genomics, genes genetic, genome, gene expression


In [48]:
dataframe = wheat.copy()
print('wheat')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/wheat.csv')

wheat
Topic #1: yield, production, productivity, yields, harvest, growth, production productivity, seasons, season, million
Topic #2: resistance, disease, diseases, pathogen, fungus, pests, virus, fungi, pest, host
Topic #3: climate, soil, climate change, environmental, gases, environment, greenhouse gas, change, mitigation, ecosystem services
Topic #4: cropping, rotation, nutrient, tillage, nitrogen, water, rotations, fertilizer, conservation, crop rotation
Topic #5: diet, students, diets, training, student, undergraduate, science, university, universities, educational
Topic #6: feed, animal, animals, livestock, biofuels, cattle, dairy, feeding, fish, biofuel
Topic #7: genes, genetic, molecular, genes genetic, gene, genome, trait, identify, genomics


In [49]:
dataframe = whole.copy()
print('whole')
dataframe['TEXT'] = dataframe['ABSTRACT'].astype(str) + dataframe[
    'PROJECT_TERMS'].astype(str) + dataframe['PROJECT_TITLE'].astype(str)
dataframe['TEXT'] = dataframe['TEXT'].astype(str)
dataframe['TEXT_PROCESSED'] = dataframe['TEXT'].map(
    lambda x: re.sub('[,\.!?]', '', x))
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].map(
    lambda x: x.lower())
dataframe['TEXT_PROCESSED'] = dataframe['TEXT_PROCESSED'].replace('\d+',
                                                                  '',
                                                                  regex=True)
vectorizer = TfidfVectorizer(max_df=.5,
                             min_df=10,
                             max_features=None,
                             ngram_range=(1, 2),
                             norm=None,
                             binary=True,
                             use_idf=False,
                             sublinear_tf=False,
                             stop_words=stopwords.words('english'))
vectorizer = vectorizer.fit(dataframe['TEXT_PROCESSED'])
tfidf = vectorizer.transform(dataframe['TEXT_PROCESSED'])
vocab = vectorizer.get_feature_names()
anchors = [[
    "production", "productivity", "yield", "yields", "growth", "harvest",
    "harvest", "index"
],
           [
               "pest", "pests", "disease", "diseases", "pesticides",
               "pesticide", "resistance", "stress", "tolerance", "fungus",
               "fungi", "virus", "pathogen", "weeds", "invasive"
           ],
           [
               "environment", "environmental", "climate", "climate change",
               "soil", "microbiome", "climate adaptation", "adaptation",
               "climate mitigation", "mitigation", "greenhouse gas", "gases",
               "ecosystem services"
           ],
           [
               "fertilizer", "nitrogen", "phosphorous", "nutrients",
               "nutrient", "nutrient management", "water", "cover crop",
               "cropping", "crop system", "crop rotation", "rotation",
               "rotations", "conservation", "small holder", "small-holder",
               "smallholder", "tillage", "till", "mulch"
           ],
           [
               "human health", "human nutrition", "nutrition", "food security",
               "food quality", "diet", "diets", "public health", "hunger",
               "plant-based", "plant based"
           ],
           [
               "livestock", "cattle", "hogs", "pigs", "chickens", "animal",
               "animals", "feed", "meat", "dairy", "milk", "biofuel",
               "biofuels","fish","aquaculture"
           ],
           [
               "economics", "cash-crop", "cash crop", "high-value",
               "high value", "investment", "market-value", "markets",
               "subsidies", "crop insurance"
           ]]
anchors = [[a for a in topic if a in vocab] for topic in anchors]
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(tfidf, words=vocab, anchors=anchors, anchor_strength=5)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))

#topic_df = pd.DataFrame(model.transform(tfidf), columns=["topic_{}".format(i+1) for i in range(7)]).astype(float)
topic_df = pd.DataFrame(model.p_y_given_x)
df = pd.concat([dataframe['TEXT'], topic_df], axis=1)
df.head(10)
df.to_csv('Topics_corex/whole.csv')

whole
Topic #1: production, yield, harvest, yields, productivity, growth, production productivity, quality, crop production
Topic #2: resistance, disease, pest, pests, pathogen, diseases, pesticides, virus, fungus, fungi
Topic #3: soil, climate, environmental, climate change, environment, gases, greenhouse gas, ecosystem services, change, agriculture
Topic #4: cropping, rotation, nitrogen, nutrient, fertilizer, water, tillage, rotations, cover crop, nutrients
Topic #5: diet, diets, nutrition, public health, foods, health, human health, human nutrition, food, nutritional
Topic #6: feed, animal, cattle, biofuels, livestock, animals, dairy, biofuel, milk, meat
Topic #7: genes, gene, genetic, genes genetic, molecular, genome, genomics, expression


In [12]:
def preprocess(data, category,corex):
    d = dict()
    df = pd.DataFrame()
    text = ''
    funding = []
    # word-count in csv
    i = 0
    for dataframe in data:
        print(category[i])
        dataframe['ABSTRACT'] = dataframe['ABSTRACT'].astype(str)
        dataframe['ABSTRACT_PROCESSED'] = dataframe['ABSTRACT'].map(
            lambda x: re.sub('[,\.!?]', '', x))
        # Convert the titles to lowercase
        dataframe['ABSTRACT_PROCESSED'] = dataframe['ABSTRACT_PROCESSED'].map(
            lambda x: x.lower())
        dataframe['ABSTRACT_PROCESSED'] = dataframe[
            'ABSTRACT_PROCESSED'].replace('\d+', '', regex=True)
        vectorizer = TfidfVectorizer(max_df=.5,
                                     min_df=10,
                                     max_features=None,
                                     ngram_range=(1, 2),
                                     norm=None,
                                     binary=True,
                                     use_idf=False,
                                     sublinear_tf=False,
                                     stop_words=stopwords.words('english'))
        vectorizer = vectorizer.fit(dataframe['ABSTRACT_PROCESSED'])
        tfidf = vectorizer.transform(dataframe['ABSTRACT_PROCESSED'])
        vocab = vectorizer.get_feature_names()

        
        anchors = [
            ["water conservation", "water efficiency"],
            ["drought"],
            ["flood"],
            ["climate", "emission", "methane", "nitrous oxide", "CH4", "N2O"],
            ["soil carbon", "soil health"],
            ["biodiversity"],
            ["yield", "productivity"],
            ["human health", "human nutrition", "human nutrients","protein"],
            ["intercrop", "relay crop","diversified"],
            ["organic","monoculture"]
        ]
        anchors = [[a for a in topic if a in vocab] for topic in anchors]
        model = ct.Corex(n_hidden=10, seed=42)
        model = model.fit(tfidf,
                          words=vocab,
                          anchors=anchors,
                          anchor_strength=3)
        for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
            topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
            print("Topic #{}: {}".format(i + 1, ", ".join(topic_ngrams)))
        topic_df = pd.DataFrame(
            model.transform(tfidf),
            columns=["topic_{}".format(i + 1) for i in range(10)]).astype(float)
        topic_df.index = dataframe.index
        dataframe= pd.concat([dataframe, topic_df], axis=1)
        i = i + 1

        print('-------')
        print('-------')


preprocess(data, category,corex)

barley
Topic #1: drought, tolerance, stress, hordeum, water, barley hordeum, drought stress, vulgare, hordeum vulgare, end use
Topic #2: climate, yellow, many, yellow dwarf, also, barley yellow, research, crops, impact, nitrogen
Topic #3: yield, productivity, yield quality, crop, high, environment, beneficial, well, however, use efficiency
Topic #4: protein, used, identified, proteins, process, regulation, measurements, activity, variability, confer
Topic #5: genetic, genes, gene, molecular, expression, genome
Topic #6: cropping systems, soil, cropping, performance, trials, field, markets, objectives, methods, rotation
Topic #7: objective, determine, characterize, host, sub objective, sub, objective determine, objective develop, interactions, mechanisms
Topic #8: management, pest, practices, aphid, pest management, farm, ipm, acres, plains, insect
Topic #9: fusarium, fusarium head, blight, head, head blight
Topic #10: germplasm, resistance, rust, new sources, identify, markers, objecti

KeyboardInterrupt: 